In [ ]:
#Import all the files to be shared among all notebooks 
import utils
import preprocessing
import data_visualization
import feature_engineering
import ML_algorithms

In [ ]:
df = utils.get_dataset()

In [ ]:
df = feature_engineering.drop_useless_columns(df)

In [ ]:
df = preprocessing.to_dtype_object(df)

In [ ]:
df = preprocessing.encode_days_as_costumer(df)

In [ ]:
df = df.dropna()

In [ ]:
data_visualization.univariate_outlier_id_plot(df)

In [ ]:
#columns = ["Year_Birth","Income","Dt_Customer","Recency","MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds","NumDealsPurchases","NumWebPurchases","NumCatalogPurchases","NumStorePurchases","NumWebVisitsMonth"]

def outlier_IQR(df, columns=["Year_Birth","Income","Dt_Customer","Recency","MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds","NumDealsPurchases","NumWebPurchases","NumCatalogPurchases","NumStorePurchases","NumWebVisitsMonth"]):
    dataframe = df[columns].copy()
    
    Q1 = dataframe.quantile(0.25)
    Q3 = dataframe.quantile(0.75)
    IQR = Q3 - Q1
       
    dataframe = dataframe[~((dataframe < (Q1 - 1.5 * IQR)) |(dataframe > (Q3 + 1.5 * IQR))).any(axis=1)]
    
    return dataframe

In [ ]:
df.shape

In [ ]:
df_IQR = outlier_IQR(df)

In [ ]:
df_IQR.dropna().shape

In [ ]:
df_IQR

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x=df_IQR["Kidhome"])
plt.show()
plt.clf()

In [ ]:
sns.boxplot(x=df["Income"])
plt.show()
plt.clf()


In [ ]:
data_visualization.univariate_outlier_id_plot(df_IQR)

In [ ]:
# IQR method
# Calculating the interquartile ranges for each column
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

# Removing all rows where a value is not within 1.5 IQR from Q1 or Q3
df_IQR = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
# With this method we would remove 14 % of our data
#(df_Merge_IQR.shape[0] -df_Merge.shape[0]) /df_Merge.shape[0]

# Less strict: Removing all rows where a value is not within 2 IQR from Q1 or Q3
#df_Merge_IQR = df_Merge[~((df_Merge < (Q1 - 2* IQR)) |(df_Merge > (Q3 + 2 * IQR))).any(axis=1)]
# This still removes 9 % of our data
#print('Removing outliers using the IQR method with 1.5 quartiles would lead to a change of data size: ',(df_Merge_IQR.shape[0] -df_Merge.shape[0]) /df_Merge.shape[0])